In [108]:
import json
import math
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
%matplotlib inline

In [109]:
with open('input21', 'r') as f:
    lines = [l.strip() for l in f.readlines()]

## Part 1

In [4]:
lines = [
    'Player 1 starting position: 4',
    'Player 2 starting position: 8',
]

In [5]:
positions = [
    int(lines[0].rsplit(' ', 1)[1]),
    int(lines[1].rsplit(' ', 1)[1])
]
scores = [0, 0]
last_dice_value = 100
num_dice_rolls = 0

cur_player = 0
while max(scores) < 1000:
    values = np.arange(last_dice_value + 1, last_dice_value + 4)
    values[values > 100] -= 100
    last_dice_value = values[-1]
    num_dice_rolls += 3
    roll = values.sum()
    positions[cur_player] += roll
    while positions[cur_player] > 10:
        positions[cur_player] -= 10
    scores[cur_player] += positions[cur_player]
    if cur_player == 0:
        cur_player = 1
    else:
        cur_player = 0
print(min(scores) * num_dice_rolls)

739785


## Part 2

In [110]:
rolls = defaultdict(int)
for i in range(1, 4):
    for j in range(1, 4):
        for k in range(1, 4):
            rolls[i+j+k] += 1

In [111]:
def do_turn(p0, p1, s0, s1, player):
    outcomes = defaultdict(int)
    for roll, universe_count in rolls.items():
        if player == 0:
            new_p0 = p0 + roll
            if new_p0 > 10:
                new_p0 -= 10
            new_s0 = s0 + new_p0
            outcomes[(new_p0, p1, new_s0, s1)] += universe_count
        else:
            new_p1 = p1 + roll
            if new_p1 > 10:
                new_p1 -= 10
            new_s1 = s1 + new_p1
            outcomes[(p0, new_p1, s0, new_s1)] += universe_count
    return outcomes

In [114]:
positions = [
    int(lines[0].rsplit(' ', 1)[1]),
    int(lines[1].rsplit(' ', 1)[1])
]

states = defaultdict(int)
states[(positions[0], positions[1], 0, 0)] = 1
player = 0
while True:
    new_states = defaultdict(int)
    for state, current_count in states.items():
        if max(state[2], state[3]) < 21:
            new_turn_states = do_turn(*state, player)
            for new_state, new_count in new_turn_states.items():
                new_states[new_state] += current_count * new_count
        else:
            new_states[state] += current_count
    states = new_states
    
    num_winning_states = 0
    for state in new_states:
        if max(state[2], state[3]) >= 21:
            num_winning_states += 1
    if num_winning_states == len(new_states):
        break
    
    if player == 0:
        player = 1
    else:
        player = 0

p1_wins = 0
p2_wins = 0
for state, universe_count in states.items():
    if state[2] > state[3]:
        p1_wins += universe_count
    else:
        p2_wins += universe_count
print(p1_wins, p2_wins)

133029050096658 74014892331523
